<a href="https://colab.research.google.com/github/ravipratap366/Complete-Deep-Learning/blob/master/PhotoGAUGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import cv2
from IPython.display import Image
import os 

In [2]:
import utlis

ModuleNotFoundError: ignored

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#!unzip '/content/drive/MyDrive/OneDrive_2022-07-14.zip' -d '/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE'

In [ ]:
#Image('/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png')

In [15]:

def getContours(img,cThr=[100,100],showCanny=False,minArea=1000,filter=0,draw =False):
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur,cThr[0],cThr[1])
    kernel = np.ones((5,5))
    imgDial = cv2.dilate(imgCanny,kernel,iterations=3)
    imgThre = cv2.erode(imgDial,kernel,iterations=2)
    if showCanny:cv2.imshow('Canny',imgThre)
    contours,hiearchy = cv2.findContours(imgThre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    finalCountours = []
    for i in contours:
        area = cv2.contourArea(i)
        if area > minArea:
            peri = cv2.arcLength(i,True)
            approx = cv2.approxPolyDP(i,0.02*peri,True)
            bbox = cv2.boundingRect(approx)
            if filter > 0:
                if len(approx) == filter:
                    finalCountours.append([len(approx),area,approx,bbox,i])
            else:
                finalCountours.append([len(approx),area,approx,bbox,i])
    finalCountours = sorted(finalCountours,key = lambda x:x[1] ,reverse= True)
    if draw:
        for con in finalCountours:
            cv2.drawContours(img,con[4],-1,(0,0,255),3)
    return img, finalCountours

def reorder(myPoints):
    #print(myPoints.shape)
    myPointsNew = np.zeros_like(myPoints)
    myPoints = myPoints.reshape((4,2))
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

def warpImg (img,points,w,h,pad=20):
    # print(points)
    points =reorder(points)
    pts1 = np.float32(points)
    pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgWarp = cv2.warpPerspective(img,matrix,(w,h))
    imgWarp = imgWarp[pad:imgWarp.shape[0]-pad,pad:imgWarp.shape[1]-pad]
    return imgWarp

def findDis(pts1,pts2):
    return ((pts2[0]-pts1[0])**2 + (pts2[1]-pts1[1])**2)**0.5

In [3]:
#getContours('/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png')

In [4]:


###################################
webcam = True
path = '/content/drive/MyDrive/Untitled folder/data_PhotoGAUGE/pib-1ltaw4xb - frames/scene00121.png'
cap = cv2.VideoCapture(0)
cap.set(10,160)
cap.set(3,1920)
cap.set(4,1080)
scale = 3
wP = 210 *scale
hP= 297 *scale
###################################

while True:
  img = cv2.imread(path)

    imgContours , conts = utlis.getContours(img,minArea=50000,filter=4)
    if len(conts) != 0:
        biggest = conts[0][2]
        #print(biggest)
        imgWarp = utlis.warpImg(img, biggest, wP,hP)
        imgContours2, conts2 = utlis.getContours(imgWarp,
                                                 minArea=2000, filter=4,
                                                 cThr=[50,50],draw = False)
        if len(conts) != 0:
            for obj in conts2:
                cv2.polylines(imgContours2,[obj[2]],True,(0,255,0),2)
                nPoints = utlis.reorder(obj[2])
                nW = round((utlis.findDis(nPoints[0][0]//scale,nPoints[1][0]//scale)/10),1)
                nH = round((utlis.findDis(nPoints[0][0]//scale,nPoints[2][0]//scale)/10),1)
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[1][0][0], nPoints[1][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05)
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[2][0][0], nPoints[2][0][1]),
                                (255, 0, 255), 3, 8, 0, 0.05)
                x, y, w, h = obj[3]
                cv2.putText(imgContours2, '{}cm'.format(nW), (x + 30, y - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 255), 2)
                cv2.putText(imgContours2, '{}cm'.format(nH), (x - 70, y + h // 2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.5,
                            (255, 0, 255), 2)
        cv2.imshow('A4', imgContours2)

    img = cv2.resize(img,(0,0),None,0.5,0.5)
    cv2.imshow('Original',img)
    cv2.waitKey(1)

IndentationError: ignored

In [2]:
import numpy as np
import pandas as pd
import cv2
from IPython.display import Image
import os 
from google.colab.patches import cv2_imshow
import uuid   # Unique identifier
import time

In [13]:
main_folder = '/content/drive/MyDrive/x'
my_folders = os.listdir(main_folder)
print(my_folders)

['3', '.ipynb_checkpoints']


In [ ]:
for folder in my_folders:
  path = main_folder +'/'+folder
  images=[]
  myList=os.listdir(path)
  print(f'total no of image detected {len(myList)}')
  for imgN in myList:
    curImg=cv2.imread(f'{path}/{imgN}')
    images.append(curImg)
  #print(len(images))
  stitcher = cv2.Stitcher.create() 
  (status,result) = stitcher.stitch(images)
  if (status == cv2.STITCHER_OK):
    print('Panorama Generated')
    cv2_imshow(result)


    IMAGES_PATH='/content/drive/MyDrive/panorama'
    # Naming out image path
    imgname = os.path.join(IMAGES_PATH,folder+str(uuid.uuid1())+'.png')

    # Writes out image to file 
    cv2.imwrite(imgname, result)
    print(imgname)
    print('Panorama saved')

    cv2.waitKey(1)
  else:
    print('panorama generation unsuccessful')

cv2.waitKey(0)



total no of image detected 3
Panorama Generated
